<a href="https://colab.research.google.com/github/diogogc/InstaCaptionCreator/blob/main/InstaCaptionCreator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criador de Legenda para o Instagram

Essa aplicação tem por objetivo aprender com suas postagens e gerar insights e ideias para Legendas para novos post.

In [ ]:
#@title Instalar Bibliotecas
# Google Gemini
!pip install -q -U google-generativeai

# API Instagram https://github.com/subzeroid/instagrapi
!pip install -q -U instagrapi

In [ ]:
#@title Libs e Variáveis
# Libs Utilizadas
from instagrapi import Client
import json
import google.generativeai as genai
from google.colab import userdata
import ipywidgets as widgets

# Dados de Acesso as API's
insta_user = userdata.get('insta_user')
insta_password = userdata.get('insta_password')
google_api_key = userdata.get('Gemini_API_KEY')

In [ ]:
#@title Obtenção dados do Insta

#@markdown Durante o processo de Login, pode ser que seja nescessário autenticar em 2 fatores. Um input irá aparecer, caso necessite.

# Classe Post com dados relevantes da postagem
class Post:
    def __init__(self, legenda, qtd_curtidas, qtd_comentarios):
        self.legenda = legenda
        self.qtd_curtidas = qtd_curtidas
        self.qtd_cometarios = qtd_comentarios

# Inicia login e acesso aos dados no Insta
# !!! Durante o processo de Login, pode ser que seja nescessário autenticar em 2 fatores. Um input irá aparecer, caso necessite. !!!
cl = Client()
cl.login(insta_user, insta_password)

user_id = cl.user_id_from_username(insta_user)


# Retorna uma Lista com as postagens
#@markdown Defina a quantidade de posts que será usada para treinar antes de executar esse trecho.

qtd_postagens = 50 #@param {type:"integer"}
medias = cl.user_medias_v1(user_id,qtd_postagens)

# Infos do Perfil
userInfo = cl.user_info_v1(user_id)

#@markdown Dados Básicos do Perfil.
print("Nome Usuario: \n",userInfo.username)
print("Bio: \n",userInfo.biography)
print("Seguidores: \n",userInfo.follower_count)

#lista para armazenar objetos do tipo Post
postList = []

# Transformando objetos do tipo Media em objetos do tipo Post para simplificar
for aMedia in medias:
    aPost = Post(aMedia.caption_text,aMedia.like_count, aMedia.comment_count)
    postList.append(aPost)

#Json da lista de objetos Post para passar ao Gemini
jsonPostList = json.dumps([aPost.__dict__ for aPost in postList])

print(jsonPostList)

In [ ]:
#@title Treinando o Google Gemini

GOOGLE_API_KEY = userdata.get('Gemini_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel(model_name="gemini-1.0-pro")

# Prompt Gerar Categorias de Post
promptLearning = "Dado o seguinte json com posts de instagram "+jsonPostList+" crie uma categorizacao dos tipos de postagem, e quero como resposta em json apenas das categorias"

chat = model.start_chat(history=[])

response = chat.send_message(promptLearning)
print(response.text)
responseCleanup = response.text.replace('```json', '')
responseCleanup = responseCleanup.replace('```', '')

# Lista de Categorias
aCategList = json.loads(responseCleanup)

print("Lista de categorias ",aCategList)


# Gemini Analizar Estilo de escrita

promptEstilo = "Analize o estilo de escrita e lembre desse estilo"

estiloResponse = chat.send_message(promptEstilo)

print("Analise das Legendas: ",estiloResponse.text)

In [ ]:
#@title Criando Prompt Geração de Legenda
#@markdown Primeiro, defina o tema ou principal conteúdo de sua postagem e depois execute esse código.

tema = 'Saturno' #@param {type:"string"}
#@markdown Após executar, selecione a categoria no drop down que dará o tom de sua postagem.
@widgets.interact(categoriaPrompt=aCategList['categorias'])

def userPrompt(categoriaPrompt):
  if len(categoriaPrompt):
    promptFinal = "Escreva um post no estilo das legendas apresentadas da categoria "+categoriaPrompt+" com tema "+tema
    return chat.send_message(promptFinal).text
  else :
    return 'Escreva um post de astronomia'

# aResponse = chat.send_message(user_prompt)
# while user_prompt != "fim":
#   aResponse = chat.send_message(user_prompt)
#   print("Resposta: ",aResponse.text, "\n")
#   user_prompt = input("Esperando Prompt: ")

In [ ]:
#@title Informações Relevantes


maisEngajaPrompt = "Legenda do post com mais Engajamento com número de curtidas e comentários "

print("Post com mais Engajamento: ",chat.send_message(maisEngajaPrompt).text)

categoriaMaisEngajaPrompt = "Qual categoria com mais engajamento"
print("Categoria com mais Engajamento: ",chat.send_message(categoriaMaisEngajaPrompt).text)
